<a href="https://colab.research.google.com/github/code-cerebrum0/ai/blob/main/Tools_in_langchan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [7]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool= DuckDuckGoSearchRun()
results = search_tool.invoke('india in sco')

print(results)

India, officially the Republic of India, [j][20] is a country in South Asia. 4 days ago · India is a country that occupies the greater part of South Asia. It is a constitutional republic that represents a highly diverse population consisting of thousands of ethnic groups. It … National Portal of India provides a single-window access to information and services that are electronically delivered from all Government Departments, Institutions and Organizations. It … 1 day ago · India in depth country profile. Unique hard to find content on India. Includes customs, culture, history, geography, economy current events, photos, video, and more. India, officially the Republic of India, [j][19] is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023; [20] and, since its independence in …


In [10]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

results

Executing command:
 whoami


/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


'root\n'

Custom Tools

In [12]:
# step1 create function

def multiply(a,b):
  """multiply two numbers"""
  return a*b

In [14]:
# step2 add type hints

def multiply(a: int, b:int) -> int:
  """Multiply two numbers"""
  return a*b

In [16]:
from langchain.tools import tool

In [17]:
#  Step 3 Add tool decorator
#due to this decorator llm can interact with this tool
@tool
def multiply(a: int, b:int) -> int:
  """Multiply two numbers"""
  return a*b


In [18]:
result = multiply.invoke({"a": 5, "b": 3})

print(result)

15


In [20]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [21]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


method 2 - Structured tool (pydantic)

In [22]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [24]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True, description = "The first number")
  b : int = Field(required = True, description = "The second number")



In [25]:
def multiply_fun(a:int, b:int) -> int:
  return a*b

In [26]:
multiply_tool = StructuredTool.from_function(
    func =  multiply_fun,
    name = "Multiply",
    description = " Multiply two numbers",
    args_schema = MultiplyInput
)

In [28]:
result = multiply_tool.invoke({'a': 3, "b": 5})
result

15

In [29]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

Multiply
Multiply two numbers
{'a': {'description': 'The first number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number', 'required': True, 'title': 'B', 'type': 'integer'}}


Method 3 Using BaseTool

In [30]:
from langchain.tools import BaseTool
from typing import Type

In [31]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True, description = "The first number")
  b : int = Field(required = True, description = "The second number")



In [32]:
class MultiplyTool(BaseTool):
  name : str = "multiply"
  description : str = "Multiply two numbers"

  args_schema: Type[BaseModel] = MultiplyInput
  def _run(self, a:int, b:int) -> int:
    return a*b

In [33]:
multiply_tool =  MultiplyTool()

In [34]:
result = multiply_tool.invoke({"a" : 8, 'b': 4})
result

32

In [35]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two numbers
{'a': {'description': 'The first number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number', 'required': True, 'title': 'B', 'type': 'integer'}}
